In [1]:
# For data handling and storage
import pandas as pd
import numpy as np
# This will prevent a misguided warning
pd.options.mode.chained_assignment = None

# For file import
import json
import os

# For text formatting
import re

# Loading the Dataset

Note: There are 7 .ldjson files stored in a folder located at ./data. There were two additional formatting steps taken first:

*  A few of the files had formatting errors (a missing line break between entries), which were manually fixed
*  The files were renamed to specify the date range and order

In [2]:
# Print the file names
files = ['1_may_july_19.ldjson',
         '2_august_19.ldjson',
         '3_july_sep_20.ldjson',
         '4_oct_dec_21.ldjson',
         '5_jan_mar_21.ldjson',
         '6_apr_june_21.ldjson',
         '7_july_sep_21.ldjson']

Each line of the data file is its own dictionary, so the entire dataset can't be loaded at once with *json.loads()*, but each line can be read as its own *.json* file. 

In [3]:
job_posts = []

# For each file
for file in files:
    print(file)
    
    # For each line of the file, read it as a .json file
    for line in open("./data/" + file, 'r'):
        job_posts.append(json.loads(line))

1_may_july_19.ldjson
2_august_19.ldjson
3_july_sep_20.ldjson
4_oct_dec_21.ldjson
5_jan_mar_21.ldjson
6_apr_june_21.ldjson
7_july_sep_21.ldjson


In [4]:
# Make it a Pandas dataframe
job_post_df = pd.DataFrame(job_posts)

# 149,904 job entries and 58 variables
print(job_post_df.shape)

# First 5 entries
job_post_df.head()

(149904, 58)


,uniq_id,crawl_timestamp,url,job_title,category,company_name,city,state,country,post_date,...,inferred_salary_time_unit,apply_url,test_contact_email,segment_name,job_desc_char_count,run_forced,ijp_is_production_ready,extra_fields,ijp_is_prod_ready,logo_url
0,f1f521b137e040af77495ef053033c1d,2019-06-15 06:58:56 +0000,https://ca.indeed.com/viewjob?jk=addd6d52ad95671a,Valet Attendant,Customer-Service,Intact,Toronto,ON,CA,2019-06-18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6782dc813048ff8feca7c07d8279e456,2019-06-20 07:33:26 +0000,https://www.indeed.com/viewjob?jk=21a20005cda7...,Office Associate (10-15 Hours Per Week) Clovis,Administrative,Transform SR LLC,Clovis,CA 93612,US,2019-06-20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8d5d487901c993190fdf129902b7cc0e,2019-05-25 08:15:32 +0000,https://www.indeed.com/viewjob?jk=3c49ca6367e2...,Line Cook,Restaurant/Food+Service,Saltgrass Parker (Denver),Parker,CO,Usa,2019-05-25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,26d36e31ed78556dc241cb418f672afc,2019-05-19 08:51:04 +0000,https://www.indeed.com/viewjob?jk=28ca41c90fa1...,Solar Installer,Construction/Facilities,Chico Electric,Chico,CA 95973,Usa,2019-05-18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8cca34bf3a41246d4e7ee76d4cbeede8,2019-05-24 08:28:23 +0000,https://www.indeed.com/viewjob?jk=6bee5df180fd...,Claims Administrator,Insurance,Brink's Incorporated,Coppell,TX 75019,Usa,2019-05-24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#job_post_df.to_pickle("./data/full_job_post_df.pkl")

# Remote Job Status

In [6]:
print(job_post_df[job_post_df['is_remote'].isna()].shape)

(30060, 58)


It turns out the first 3 datasets don't have an is_remote value, so those 30,060 entries will be removed. The dataset of jobs with an is_remote value has 119,844 entries.

In [7]:
# Subset entries that do *not* have an NA value for is_remote
remote_post_df = job_post_df[-job_post_df['is_remote'].isna()]

# Since some entries were removed, the indices need to be reset
remote_post_df = remote_post_df.reset_index(drop = True)

print(remote_post_df.shape)
remote_post_df.head()

(119844, 58)


,uniq_id,crawl_timestamp,url,job_title,category,company_name,city,state,country,post_date,...,inferred_salary_time_unit,apply_url,test_contact_email,segment_name,job_desc_char_count,run_forced,ijp_is_production_ready,extra_fields,ijp_is_prod_ready,logo_url
0,19fdcca8fe64fd62515e8e07d98290e4,2020-11-10 22:36:39 +0000,https://www.indeed.com/viewjob?jk=1db200d5d41a...,Material Handler,Transportation-or-logistics,Axium Plastics,Ontario,CA,US,2020-11-10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,c9b16a4a7b1fe7bbf1cbf25dcc2f4031,2020-11-11 09:41:39 +0000,https://www.indeed.com/viewjob?jk=208982b29f90...,Full Time HHA/CNA/PCA Position,Healthcare,Right at Home of the Triangle,Raleigh,NC,US,2020-11-11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,082627b3ebd5ececaeef450318769d72,2020-10-17 14:52:34 +0000,https://www.indeed.com/viewjob?jk=2bd33ce4b684...,Senior Retail Supervisor,Retail,Goodwill Industries Of Delaware & D,Holmes,PA,US,2020-10-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,89141ac20e755763933dffa93ef6abc5,2020-11-30 07:18:25 +0000,https://www.indeed.com/viewjob?jk=b8d4b53c0647...,Medical Biller / Collector (Wilmington),Administrative,Carolina Dunes Behavioral Health,Wilmington,NC,US,2020-11-30,...,hourly,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,62206e5ae69b88bcb0c4f22f3da39dae,2020-11-04 17:58:54 +0000,https://www.indeed.com/viewjob?jk=5b5e21af1923...,Outside Sales Professional,Sales,Terminix,Auburn,AL,US,2020-11-04,...,NaN,https://servicemaster.wd1.myworkdayjobs.com/en...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# For categorical variables, prints the number of rows in each category
remote_post_df['is_remote'].value_counts()

false    113904
true       5940
Name: is_remote, dtype: int64

The dataframe has 113,904 non-remote entries and 5,940 remote ones. This difference is a little concerning for prediction purposes.

In [9]:
remote_post_df.to_pickle("./data/remote_job_post_df.pkl")

# Preparing the "State" column

The dataframe has *state*, *test1_states*, and *inferred_state* values. The *inferred_state* column seems to have been edited by the data collectors, removing strangely formatted values and adding the *state* when it is accidentally in the *city* column.

The only downside is that remote jobs are listed as NA, so for the purposes of this analysis we will add a "remote" value to *inferred_state*. 

In [10]:
no_state = remote_post_df.loc[remote_post_df['state'].isna(), ["city", "state", "test1_states", "inferred_state"]]

In [11]:
# For each entry with an NA value for state
for index in no_state.index:
    
    # If the city or state is listed as "Remote", change the value in the original dataframe
    if no_state.at[index, "city"] == "Remote":
        remote_post_df.at[index, "inferred_state"] = "Remote"
    elif no_state.loc[index, "state"] == "Remote":
        remote_post_df.at[index, "inferred_state"] = "Remote"

In [12]:
# There are NaN values and "" (empty) values
# To make all empty values uniform, NaN values will be set to ""
remote_post_df.loc[remote_post_df['inferred_state'].isna(), "inferred_state"] = ""

In [13]:
remote_post_df['inferred_state'].unique()

array(['California', 'North carolina', 'Pennsylvania', 'Alabama', '',
       'Florida', 'New york', 'Louisiana', 'Ohio', 'Colorado',
       'New jersey', 'Illinois', 'Texas', 'Arizona', 'Wisconsin',
       'Kansas', 'Michigan', 'Nevada', 'Minnesota', 'Kentucky',
       'Tennessee', 'Oklahoma', 'Indiana', 'Washington', 'Virginia',
       'New mexico', 'Maine', 'Iowa', 'Connecticut', 'Idaho',
       'West virginia', 'Oregon', 'Alaska', 'Massachusetts',
       'New hampshire', 'Maryland', 'Missouri', 'Arkansas', 'Georgia',
       'Utah', 'Nebraska', 'District of columbia', 'Delaware',
       'Rhode island', 'Mississippi', 'South dakota', 'Montana',
       'North dakota', 'South carolina', 'Hawaii', 'Vermont', 'Remote',
       'Wyoming'], dtype=object)

In the end, there are 50 states + DC, Remote, and NA values

# Preparing the "Category" column

One way to look at the type of job is the "category" feature. 

### Re-formatting "category"

It turns out that for some inexplicable reason, job categories are recorded in two different formats, for example *Computer-or-internet* and *Computer/internet*. This step standardizes the way they are written by just separating all words with a space.

In [14]:
# Starts off with 45 categories
print(len(remote_post_df['category'].unique()))

45


In [15]:
# Will use a dictionary to match category names to a generic form
job_dict = {}

# For each job type
for job_type in remote_post_df['category'].unique():
    
    # Split words on anything that isn't a letter (so / or -)
    split = re.split(r'\W', job_type)
    
    # Removing the word "or"
    split = [word for word in split if word != "or"]
    
    # Combine all the words left in the list into a string, separated only by spaces
    new = " ".join(split)
    
    # Assign old job label as key, and new uniform job label as value
    job_dict[job_type] = new

In [16]:
print("Input: Computer-or-internet")
print("Output: " + job_dict["Computer-or-internet"])

print("")

print("Input: Computer/internet")
print("Output: " + job_dict["Computer/internet"])

Input: Computer-or-internet
Output: Computer internet

Input: Computer/internet
Output: Computer internet


In [17]:
# Apply this dictionary to the entire category column
remote_post_df['category'] = remote_post_df['category'].apply(lambda x: job_dict[x])

In [18]:
# Only 27 different categories now
print(len(remote_post_df['category'].unique()))

27


### Making dummy variables

An exhaustive best subset selection run in R indicated these 17 categories were the most useful predictors, so (like in a one vs. all analysis), I will make 17 columns, each one indicating if the job category is (1) or is not (0) in a specific category. 

In [19]:
categories_orig = ["categoryAdministrative", "categoryArts entertainment publishing", "categoryComputer internet", "categoryConstruction facilities", "categoryEducation training", "categoryEngineering architecture", "categoryHealthcare", "categoryHospitality travel", "categoryLaw Enforcement security", "categoryLegal", "categoryManufacturing mechanical", "categoryMarketing advertising pr", "categoryPharmaceutical bio tech", "categoryReal Estate", "categoryRestaurant food Service", "categoryRetail", "categoryTransportation logistics"]

# This what the categories look like formatted from R
print(categories_orig[0:3])

['categoryAdministrative', 'categoryArts entertainment publishing', 'categoryComputer internet']


In [20]:
# This entire code block reformats the category names again to be column names
# This part came well after the first re-formatting in the process
# It wasn't just reformatted twice for fun

print("Reformatting process: \n")

categories_orig = [word[8:] for word in categories_orig]
print(categories_orig[1])

categories = [word.lower().split() for word in categories_orig]
print(categories[1])

categories = ["_".join(word) for word in categories]
print(categories[1])

categories = ["is_" + word for word in categories]
print(categories[1])

Reformatting process: 

Arts entertainment publishing
['arts', 'entertainment', 'publishing']
arts_entertainment_publishing
is_arts_entertainment_publishing


In [21]:
#### Making it into a dataframe ####

# This will end up having 17 more columns, so it will start with only a few columns from the original df
category_df = remote_post_df[["uniq_id", "is_remote", "post_date", "category", "inferred_state"]]

# Each category will have a column
# Within each column, the row will have a 1 if the category is the one of interest and 0 otherwise
for i in range(0, len(categories)):
    category_df[categories[i]] = category_df.apply(lambda x: 1 if x.category == categories_orig[i] else 0, axis = 1)
    
category_df.head()

,uniq_id,is_remote,post_date,category,inferred_state,is_administrative,is_arts_entertainment_publishing,is_computer_internet,is_construction_facilities,is_education_training,...,is_hospitality_travel,is_law_enforcement_security,is_legal,is_manufacturing_mechanical,is_marketing_advertising_pr,is_pharmaceutical_bio_tech,is_real_estate,is_restaurant_food_service,is_retail,is_transportation_logistics
0,19fdcca8fe64fd62515e8e07d98290e4,false,2020-11-10,Transportation logistics,California,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,c9b16a4a7b1fe7bbf1cbf25dcc2f4031,false,2020-11-11,Healthcare,North carolina,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,082627b3ebd5ececaeef450318769d72,false,2020-10-17,Retail,Pennsylvania,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,89141ac20e755763933dffa93ef6abc5,false,2020-11-30,Administrative,North carolina,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,62206e5ae69b88bcb0c4f22f3da39dae,false,2020-11-04,Sales,Alabama,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
#category_df.to_pickle("./data/remote_with_categories.pkl")

# Salary

Similar to *state*, there are multiple *salary* variables.

In [23]:
remote_post_df.loc[0:5, ["salary_offered", 'inferred_salary_currency', 'inferred_salary_from', 'inferred_salary_to', 'inferred_salary_time_unit']]

,salary_offered,inferred_salary_currency,inferred_salary_from,inferred_salary_to,inferred_salary_time_unit
0,NaN,USD,NaN,NaN,NaN
1,NaN,USD,NaN,NaN,NaN
2,NaN,USD,NaN,NaN,NaN
3,Pay: $16.00 - $18.00 per hour,USD,16.0,18.0,hourly
4,NaN,USD,NaN,NaN,NaN
5,NaN,USD,NaN,NaN,NaN


The best salary measure is the mean of *inferred_salary_from* and *inferred_salary_to*, because they are formatted.

In [24]:
# Subset that has a salary range
print(remote_post_df[-remote_post_df['inferred_salary_from'].isna()].shape)

(30234, 58)


Unfortunately, only 30,234 job entries have a salary. This is a really small sample, so it will be its own dataframe.

In [25]:
# Subset if there is a salary, reset index
has_salary_df = remote_post_df.loc[-remote_post_df['inferred_salary_from'].isna(), ["uniq_id", "is_remote", "post_date", "category", "inferred_state", "inferred_salary_from", "inferred_salary_to", "inferred_salary_time_unit"]]
has_salary_df = has_salary_df.reset_index(drop = True)

### Standardizing Salary Unit

The salaries come in different units (i.e. hourly vs. yearly), so they will be standardized to per hour.

In [26]:
def salary_format(low_range, high_range, unit):
    
    # Accounting for NA values in the high and low range values
    if np.isnan(high_range):
        high_range = low_range
    if np.isnan(low_range):
        low_range = high_range
    
    # Accounting for NA values in the unit (estimated)
    if type(unit) != str:
        # Likely hourly, $50/hour is pretty high I think but too low for weekly
        if low_range < 50:
            unit = "hourly"
        # Probably too high for monthly, low range for yearly
        elif low_range > 20000:
            unit = "yearly"
    
    # Standardizing pay unit to hourly
    if unit == "yearly":
        low_range = low_range / 2080
        high_range = high_range / 2080
    elif unit == "weekly":
        low_range = low_range / 40
        high_range = high_range / 40
    elif unit == "monthly":
        # Value based on average work days/month
        low_range = low_range / 163.256
        high_range = high_range / 163.256
    elif unit == "daily":
        low_range = low_range / 8
        high_range = high_range / 8
    
    # Find the middle of the range and return
    average = (high_range + low_range) / 2
    return average

In [27]:
# Standardize and average salary
has_salary_df['mean_salary_range'] = has_salary_df.apply(lambda x: salary_format(x.inferred_salary_from, x.inferred_salary_to, x.inferred_salary_time_unit) , axis = 1)

# Drop the low and high range variables
has_salary_df = has_salary_df.drop("inferred_salary_from", axis = 1).drop("inferred_salary_to", axis = 1)

has_salary_df.head()

,uniq_id,is_remote,post_date,category,inferred_state,inferred_salary_time_unit,mean_salary_range
0,89141ac20e755763933dffa93ef6abc5,false,2020-11-30,Administrative,North carolina,hourly,17.0
1,c212c1daf14d0308e3d91d9511f3fa32,false,2020-10-08,Healthcare,New york,hourly,20.0
2,4e3c1d579b134e4619cfd7a0dd456add,false,2020-10-30,Education training,Wisconsin,hourly,13.5
3,0e58cf674b73a60057a21ba3caca370e,false,2020-12-11,Accounting finance,,hourly,32.5
4,03125df2b73081dd1938842018f64af7,false,2020-10-25,Transportation logistics,Michigan,hourly,15.0


In [28]:
has_salary_df.to_pickle("./data/has_salary_df.pkl")

# Making a proportional dataframe

The next step is converting the unit of analysis from the entry-level to the day-level. This dataframe will be used for the model predicting the proportion of entries that are remote for each day.

In [29]:
# Get a list of all dates in the dataset in order
all_dates = remote_post_df['post_date'].unique()
all_dates.sort()

print("First Day: " + str(all_dates[0]))
print("Last Day: " + str(all_dates[-1]))

First Day: 2020-09-30
Last Day: 2021-09-30


In [30]:
proportion = []
is_remote = []
not_remote = []

# Will iterate through all of the days
for date in all_dates:
    
    # Takes only the rows in which the post_date is the current date
    subset = remote_post_df[remote_post_df['post_date'] == date]
    
    # Finds the number if remote and non-remote jobs on that day
    remote = subset['is_remote'].value_counts()
    
    false = 0
    true = 0
    
    # If there are "false" (non-remote) jobs, set false to the number of non-remote jobs
    if 'false' in remote.index:
        false = remote['false']
    
    # If there are "true" (remote) jobs, true = the number of remote jobs
    if 'true' in remote.index:
        true = remote['true']
    
    # Store the number of remote and non-remote jobs
    is_remote.append(true)
    not_remote.append(false)
    
    # For this one day, the proportion is the number of remote jobs / total jobs
    proportion.append(true / (true + false))

In [31]:
by_day = pd.DataFrame(list(zip(all_dates, is_remote, not_remote, proportion)), columns = ["date", "is_remote", "not_remote", "prop_remote"])
by_day.head()

,date,is_remote,not_remote,prop_remote
0,2020-09-30,0,4,0.000000
1,2020-10-01,19,427,0.042601
2,2020-10-02,23,503,0.043726
3,2020-10-03,29,463,0.058943
4,2020-10-04,3,157,0.018750


In [32]:
by_day.to_pickle("./data/prop_by_day.pkl")